<a href="https://colab.research.google.com/github/amaan-zafar/topic-modelling/blob/main/2_DataCleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import re
import pandas as pd
from pprint import pprint
import string

# NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Spacy
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [27]:
# Imporing json files
import json
from google.colab import files
uploaded = files.upload()

Saving thehindu_jan_11_file_1.json to thehindu_jan_11_file_1.json
Saving thehindu_jan_11_file_2.json to thehindu_jan_11_file_2.json
Saving thehindu_jan_11_file_3.json to thehindu_jan_11_file_3.json
Saving thehindu_jan_11_file_4.json to thehindu_jan_11_file_4.json
Saving thehindu_jan_11_file_5.json to thehindu_jan_11_file_5.json
Saving thehindu_jan_11_file_6.json to thehindu_jan_11_file_6.json
Saving thehindu_jan_11_file_7.json to thehindu_jan_11_file_7.json
Saving thehindu_jan_11_file_10.json to thehindu_jan_11_file_10.json
Saving thehindu_jan_11_file_12.json to thehindu_jan_11_file_12.json
Saving thehindu_jan_11_file_13.json to thehindu_jan_11_file_13.json
Saving thehindu_jan_11_file_14.json to thehindu_jan_11_file_14.json
Saving thehindu_jan_11_file_15.json to thehindu_jan_11_file_15.json
Saving thehindu_jan_11_file_16.json to thehindu_jan_11_file_16.json
Saving thehindu_jan_11_file_17.json to thehindu_jan_11_file_17.json
Saving thehindu_jan_11_file_18.json to thehindu_jan_11_file_18

In [28]:
json_files = list(uploaded.keys())
json_files

['thehindu_jan_11_file_1.json',
 'thehindu_jan_11_file_2.json',
 'thehindu_jan_11_file_3.json',
 'thehindu_jan_11_file_4.json',
 'thehindu_jan_11_file_5.json',
 'thehindu_jan_11_file_6.json',
 'thehindu_jan_11_file_7.json',
 'thehindu_jan_11_file_10.json',
 'thehindu_jan_11_file_12.json',
 'thehindu_jan_11_file_13.json',
 'thehindu_jan_11_file_14.json',
 'thehindu_jan_11_file_15.json',
 'thehindu_jan_11_file_16.json',
 'thehindu_jan_11_file_17.json',
 'thehindu_jan_11_file_18.json',
 'thehindu_jan_11_file_19.json',
 'thehindu_jan_11_file_21.json',
 'thehindu_jan_11_file_23.json',
 'thehindu_jan_11_file_24.json',
 'thehindu_jan_11_file_27.json',
 'thehindu_jan_11_file_28.json',
 'thehindu_jan_11_file_29.json',
 'thehindu_jan_11_file_30.json',
 'thehindu_jan_11_file_31.json',
 'thehindu_jan_11_file_32.json',
 'thehindu_jan_11_file_33.json',
 'thehindu_jan_11_file_34.json',
 'thehindu_jan_11_file_35.json',
 'thehindu_jan_11_file_36.json',
 'thehindu_jan_11_file_37.json',
 'thehindu_jan_11

In [29]:
corpus = []
for file_name in json_files:
  decoded_json = json.loads(uploaded[file_name].decode("utf-8"))
  corpus.append(''.join(decoded_json['text']))
corpus[0]

'\nDivyansh pips Rudrankksh for top spot\n\nKavi Rakshna takes the honours in the women’s air rifle\nWorld No. 1 in men’s air rifle, Divyansh Singh Panwar asserted his class by rising to the top position in the second competition of the National shooting selection trials on Sunday.Qualifiying in the eighth place with 628.7 points, a mere 0.1 point ahead of Srinjoy Datta, Divyansh shot 250.9 in the final, to beat Rudrankksh Patil by 0.8 point for the top spot.Hazarika thirdWorld junior champion Hriday Hazarika, who had won the first trial, was pushed to the third spot. Youth Olympics silver medallist Shahu Tushar Mane placed fourth while qualification topper Arjun Babuta (632.1) finished seventh.In the women’s section, Kavi Rakshna of Tamil Nadu emerged on top with 251.4, as she beat qualification topper Nisha Kanwar (630.7) by 0.7 point. Remarkably, Kavi had risen to the top spot after having placed seventh in qualification. World No. 1 and winner of first trial, Elavenil Valarivan pla

In [31]:
df = pd.DataFrame(corpus, columns = {'Raw_Text'})
df

,Raw_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...
1,\nLombardy proves her class in HRC Golconda 10...
2,"\nVictory Parade, Race For The Stars, Demerara..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...
...,...
326,\nData | MSMEs continue to be most stressed se...
327,"\nAmong the countries that began inoculation, ..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...
329,"\nRenowned writer Ved Mehta, who took India to..."


**Data Cleaning**

In [57]:
# Removing extra whitespaces (Removes line characters too)
def remove_extra_whitespace(text):
  return re.sub("\s+"," ", text)

# Remove punctuations and convert to lower case
def lower_without_punct(text):
  lower_no_punct = "".join([i.lower() for i in text if i not in string.punctuation])
  words_wo_punct = ''.join(lower_no_punct)
  return words_wo_punct

def remove_numbers(text):
  return ''.join([i for i in text if not i.isdigit()])

cleaned_corpus = []
for doc in corpus:
  wo_extra_ws = remove_extra_whitespace(doc)
  lower_wo_punct = lower_without_punct(wo_extra_ws)
  wo_numbers = remove_numbers(lower_wo_punct)
  cleaned_corpus.append(wo_numbers)

cleaned_corpus[0]

' divyansh pips rudrankksh for top spot kavi rakshna takes the honours in the women’s air rifle world no  in men’s air rifle divyansh singh panwar asserted his class by rising to the top position in the second competition of the national shooting selection trials on sundayqualifiying in the eighth place with  points a mere  point ahead of srinjoy datta divyansh shot  in the final to beat rudrankksh patil by  point for the top spothazarika thirdworld junior champion hriday hazarika who had won the first trial was pushed to the third spot youth olympics silver medallist shahu tushar mane placed fourth while qualification topper arjun babuta  finished seventhin the women’s section kavi rakshna of tamil nadu emerged on top with  as she beat qualification topper nisha kanwar  by  point remarkably kavi had risen to the top spot after having placed seventh in qualification world no  and winner of first trial elavenil valarivan placed thirdin the second skeet trials amrinder singh cheema pippe

**Tokenization**

In [117]:
corpus_wo_persons = []
all_person_list = []
nlp = en_core_web_sm.load()
for doc in corpus:
  text = nlp(doc)
  tokens = []
  person_list = []

  for x in text.ents:
    if (x.label_=='PERSON'):
      person_list = person_list + x.text.split()

  wo_persons = [word for word in doc.split() if word not in person_list]
  all_person_list = all_person_list + person_list
  corpus_wo_persons.append(wo_persons)     

In [118]:
all_person_list

['Srinjoy',
 'Datta',
 'Divyansh',
 'Hazarika',
 'Shahu',
 'Tushar',
 'Mane',
 'Arjun',
 'Babuta',
 'Nisha',
 'Kanwar',
 'Elavenil',
 'Valarivan',
 'Amrinder',
 'Singh',
 'Cheema',
 'Fatehbir',
 'Singh',
 'Shergill',
 'Anantjeet',
 'Singh',
 'Naruka',
 'Gurjoat',
 'Singh',
 'Raiza',
 'Dhillon',
 'Divyansh',
 'Singh',
 'Hazarika',
 'Nisha',
 'Kanwar',
 '250.7',
 'Amrinder',
 'Singh',
 'Cheema',
 '54(2',
 'Fatehbir',
 'Singh',
 'Gurjoat',
 'Singh',
 '43',
 'Raiza',
 'Dhillon',
 'Areeba',
 'Khan',
 '39',
 'Abhay',
 'Singh',
 'Sekhon',
 'Areeba',
 'Khan',
 '50',
 'Raiza',
 'Dhillon',
 'Lombardy',
 'Jan.11',
 'Simone',
 'Zavaray',
 'Rishad',
 'N.',
 'Pandole',
 'Aditya',
 'P.',
 'Thackersey',
 'Lombardy',
 'Letir',
 'Mor',
 'A.',
 'A.',
 'Vikrant',
 'Afroz',
 'Khan',
 'Tanala',
 'Teja',
 'Gollapudi',
 'Ashad',
 'Asbar',
 'Akshay',
 'Kumar',
 'Golden',
 'Lady',
 'Muaser',
 'Tanala',
 'Ram',
 'H.',
 'Raj',
 'Shroff',
 'Chetan',
 'Shantilal',
 'Shah',
 'M.',
 'Srinivas',
 'BLAZER',
 'Akshay',


In [119]:
corpus_wo_persons

[['pips',
  'Rudrankksh',
  'for',
  'top',
  'spot',
  'Kavi',
  'Rakshna',
  'takes',
  'the',
  'honours',
  'in',
  'the',
  'women’s',
  'air',
  'rifle',
  'World',
  'No.',
  '1',
  'in',
  'men’s',
  'air',
  'rifle,',
  'Panwar',
  'asserted',
  'his',
  'class',
  'by',
  'rising',
  'to',
  'the',
  'top',
  'position',
  'in',
  'the',
  'second',
  'competition',
  'of',
  'the',
  'National',
  'shooting',
  'selection',
  'trials',
  'on',
  'Sunday.Qualifiying',
  'in',
  'the',
  'eighth',
  'place',
  'with',
  '628.7',
  'points,',
  'a',
  'mere',
  '0.1',
  'point',
  'ahead',
  'of',
  'Datta,',
  'shot',
  '250.9',
  'in',
  'the',
  'final,',
  'to',
  'beat',
  'Rudrankksh',
  'Patil',
  'by',
  '0.8',
  'point',
  'for',
  'the',
  'top',
  'spot.Hazarika',
  'thirdWorld',
  'junior',
  'champion',
  'Hriday',
  'Hazarika,',
  'who',
  'had',
  'won',
  'the',
  'first',
  'trial,',
  'was',
  'pushed',
  'to',
  'the',
  'third',
  'spot.',
  'Youth',
  'Olym

In [120]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(corpus_wo_persons, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

# Define functions for stopwords, bigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [128]:
# Remove Stop Words
data_words_nostops = remove_stopwords(corpus_wo_persons)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

df['Processed_Text'] = data_words_bigrams
df

,Raw_Text,Processed_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...,"[pips, rudrankksh, top, spot, kavi, rakshna, t..."
1,\nLombardy proves her class in HRC Golconda 10...,"[proves, class, hrc, golconda, guineas, traine..."
2,"\nVictory Parade, Race For The Stars, Demerara...","[victory, parade, race, stars, demerara, pirat..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...,"[hrc, golconda, guineas, four, year, old, fill..."
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...,"[isro, adopt, atal, tinkering, labs, promote, ..."
...,...,...
326,\nData | MSMEs continue to be most stressed se...,"[continue, stressed, sector, urban, cooperativ..."
327,"\nAmong the countries that began inoculation, ...","[among, countries, began, inoculation, pfizer,..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...,"[ashok, non, cookbook, kitchen, nerds, looks, ..."
329,"\nRenowned writer Ved Mehta, who took India to...","[renowned, writer, mehta, took, india, america..."


In [129]:
from nltk.stem import WordNetLemmatizer  
lemmatizer = WordNetLemmatizer()

for m in range(len(json_files)):
  temp1 = []
  temp2 = df['Processed_Text'].values[m]
  for i in temp2:
    temp1.append(lemmatizer.lemmatize(i))
  df['Processed_Text'].values[m] = temp1 
df

,Raw_Text,Processed_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...,"[pip, rudrankksh, top, spot, kavi, rakshna, ta..."
1,\nLombardy proves her class in HRC Golconda 10...,"[prof, class, hrc, golconda, guinea, trainer, ..."
2,"\nVictory Parade, Race For The Stars, Demerara...","[victory, parade, race, star, demerara, pirate..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...,"[hrc, golconda, guinea, four, year, old, filly..."
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...,"[isro, adopt, atal, tinkering, lab, promote, s..."
...,...,...
326,\nData | MSMEs continue to be most stressed se...,"[continue, stressed, sector, urban, cooperativ..."
327,"\nAmong the countries that began inoculation, ...","[among, country, began, inoculation, pfizer, b..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...,"[ashok, non, cookbook, kitchen, nerd, look, sc..."
329,"\nRenowned writer Ved Mehta, who took India to...","[renowned, writer, mehta, took, india, america..."


In [130]:
for i in range(len(json_files)):
  temp1 = []
  temp2 = df['Processed_Text'].values[i]
  for j in temp2:
    if (len(j)>2):
      temp1.append(j)
  df['Processed_Text'].values[i] = temp1 
df

,Raw_Text,Processed_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...,"[pip, rudrankksh, top, spot, kavi, rakshna, ta..."
1,\nLombardy proves her class in HRC Golconda 10...,"[prof, class, hrc, golconda, guinea, trainer, ..."
2,"\nVictory Parade, Race For The Stars, Demerara...","[victory, parade, race, star, demerara, pirate..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...,"[hrc, golconda, guinea, four, year, old, filly..."
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...,"[isro, adopt, atal, tinkering, lab, promote, s..."
...,...,...
326,\nData | MSMEs continue to be most stressed se...,"[continue, stressed, sector, urban, cooperativ..."
327,"\nAmong the countries that began inoculation, ...","[among, country, began, inoculation, pfizer, b..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...,"[ashok, non, cookbook, kitchen, nerd, look, sc..."
329,"\nRenowned writer Ved Mehta, who took India to...","[renowned, writer, mehta, took, india, america..."


In [ ]:
df.to_csv("Jan14.csv",index = True, header=True)